In [1]:
!pip install -U openai-whisper -q
!apt install ffmpeg -y -q # -y for auto-yes to prompts
!pip install yt-dlp -q
!pip install gspread pandas -q # Ensure gspread and pandas are installed

import yt_dlp
import whisper
import os
import re
import json
import gspread
import pandas as pd
from google.colab import auth
from google.auth import default
from pathlib import Path

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 19.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 68.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 31.8 MB/s

In [4]:
def extract_basic_segments(result):
    """
    Extracts basic segment information (id, start, end, text) from Whisper's transcription result.
    """
    return [
        {
            "id": seg.get("id"),
            "start": seg.get("start"),
            "end": seg.get("end"),
            "text": seg.get("text", "").strip(),
        }
        for seg in result.get("segments", [])
    ]

def get_youtube_urls_from_sheet(spreadsheet_key: str) -> list:
    """
    Authenticates with Google and retrieves YouTube URLs from a specified Google Sheet.

    Args:
        spreadsheet_key (str): The key of the Google Spreadsheet.

    Returns:
        list: A list of YouTube URLs.
    """
    print("Authenticating with Google Sheets...")
    try:
        auth.authenticate_user()
        creds, _ = default()
        gc = gspread.authorize(creds)
        spreadsheet = gc.open_by_key(spreadsheet_key)
        worksheet = spreadsheet.get_worksheet(0) # Assuming URLs are in the first worksheet

        # Get all values and convert to DataFrame
        df = pd.DataFrame(worksheet.get_all_values())
        if df.empty:
            print("Error: Google Sheet is empty.")
            return []

        # Assuming the first row is headers and 'youtube Url' is the column name
        df.columns = df.iloc[0]
        df = df.drop(0).reset_index(drop=True)

        if "youtube Url" not in df.columns:
            print("Error: 'youtube Url' column not found in Google Sheet.")
            return []

        urls = df["youtube Url"].dropna().tolist()
        print(f"Found {len(urls)} YouTube URLs in the spreadsheet.")
        return urls

    except Exception as e:
        print(f"Error accessing Google Sheet: {e}")
        return []

# --- 4. Function to Download Audio Directly from YouTube ---
def download_audio(youtube_url: str, output_dir: Path) -> Path | None:
    """
    Downloads the audio stream from a YouTube video directly to a WAV file.

    Args:
        youtube_url (str): The URL of the YouTube video.
        output_dir (Path): The directory where the audio file will be saved.

    Returns:
        Path | None: The path to the downloaded WAV file, or None if download fails.
    """
    # Sanitize URL for filename (yt_dlp usually handles this, but good for consistency)
    # Get a safe title from yt_dlp info without downloading the whole thing yet
    try:
        with yt_dlp.YoutubeDL({'quiet': True, 'extract_flat': True, 'force_generic_extractor': True}) as ydl:
            info = ydl.extract_info(youtube_url, download=False)
            video_title = info.get('title', 'unknown_video')
            # Simple regex to remove invalid characters for filenames
            safe_title = re.sub(r'[^\w\-_.]', '_', video_title)
    except Exception as e:
        print(f"Could not get video title for {youtube_url}: {e}. Using generic name.")
        safe_title = "unknown_video_" + re.sub(r'[^\w]', '', youtube_url)[:10] # Fallback

    output_path = output_dir / f"{safe_title}.wav"

    ydl_opts = {
        'format': 'bestaudio/best', # Prioritize best audio
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'wav', # Extract as WAV
            'preferredquality': '192', # High quality audio
        }],
        'outtmpl': str(output_path.with_suffix('')), # Output template without extension, postprocessor adds it
        'noplaylist': True, # Only download single video, not playlist
        'quiet': True, # Suppress console output
        'noprogress': True, # Suppress progress bar
        'restrictfilenames': True, # Ensure safe filenames
        'cookiefile': '/content/www.youtube.com_cookies.txt', # Use cookies if available
    }

    print(f"Downloading audio for: {youtube_url}")
    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([youtube_url])
        if output_path.exists():
            print(f"Successfully downloaded audio to: {output_path}")
            return output_path
        else:
            print(f"Error: Audio file not found after download attempt for {youtube_url}.")
            return None
    except yt_dlp.utils.DownloadError as e:
        print(f"Error downloading audio from {youtube_url}: {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred during audio download for {youtube_url}: {e}")
        return None

# --- 5. Function to Transcribe Audio with Whisper ---
def transcribe_audio(audio_path: Path, whisper_model) -> list | None:
    """
    Transcribes an audio file using the loaded Whisper model.

    Args:
        audio_path (Path): The path to the audio file.
        whisper_model: The loaded Whisper model instance.

    Returns:
        list | None: A list of transcription segments, or None if transcription fails.
    """
    print(f"Transcribing audio: {audio_path}")
    try:
        # Using verbose=False to suppress detailed transcription logs unless needed
        result = whisper_model.transcribe(str(audio_path), verbose=False, temperature=0.4, condition_on_previous_text=True)
        segments = extract_basic_segments(result)
        print(f"Transcription complete for: {audio_path}")
        return segments
    except Exception as e:
        print(f"Error transcribing {audio_path}: {e}")
        return None

# --- 6. Function to Save Transcription to JSON ---
def save_transcription_to_json(segments: list, output_dir: Path, original_filename: str):
    """
    Saves transcription segments to a JSON file.

    Args:
        segments (list): The list of transcription segments.
        output_dir (Path): The directory where the JSON file will be saved.
        original_filename (str): The original filename (e.g., video title) to base the JSON filename on.
    """
    json_filename = output_dir / f"{Path(original_filename).stem}_segments.json"
    print(f"Saving segments to: {json_filename}")

    data_to_save = {
        "filename": f"{Path(original_filename).stem}",
        "segments": segments
    }

    try:
        with open(json_filename, "w", encoding="utf-8") as f:
            json.dump(data_to_save, f, ensure_ascii=False, indent=2)
        print(f"Successfully saved segments to {json_filename}")
    except Exception as e:
        print(f"Error saving segments to {json_filename}: {e}")

In [5]:
# --- 7. Main Workflow Execution ---
def main_workflow(spreadsheet_key: str, audio_output_folder: str = "/content/audioFiles", transcription_output_folder: str = "/content/subtitles"):
    """
    Orchestrates the entire process: fetches URLs, downloads audio, transcribes, and saves.

    Args:
        spreadsheet_key (str): The key of the Google Spreadsheet containing YouTube URLs.
        audio_output_folder (str): Directory to save temporary audio files.
        transcription_output_folder (str): Directory to save final JSON transcriptions.
    """
    # Create output directories if they don't exist
    audio_dir = Path(audio_output_folder)
    transcription_dir = Path(transcription_output_folder)
    audio_dir.mkdir(parents=True, exist_ok=True)
    transcription_dir.mkdir(parents=True, exist_ok=True)

    # Load Whisper model once
    print("Loading Whisper model (turbo)... This may take a moment.")
    try:
        # Note: "turbo" is not a standard Whisper model size. Assuming it's a custom
        # or a very small/fast model. Standard sizes are 'tiny', 'base', 'small', 'medium', 'large'.
        # If 'turbo' fails, try 'base' or 'small'.
        whisper_model = whisper.load_model("base", download_root="/content/Data/Models")
        print("Whisper model loaded.")
    except Exception as e:
        print(f"Error loading Whisper model: {e}. Please check model name or internet connection.")
        return

    youtube_urls = get_youtube_urls_from_sheet(spreadsheet_key)

    if not youtube_urls:
        print("No YouTube URLs to process. Exiting.")
        return

    for i, url in enumerate(youtube_urls):
        print(f"\n--- Processing video {i+1}/{len(youtube_urls)}: {url} ---")
        audio_file_path = None # Initialize to None

        try:
            # Step 1: Download audio directly
            audio_file_path = download_audio(url, audio_dir)
            if audio_file_path is None:
                print(f"Skipping transcription for {url} due to download failure.")
                continue

            # Step 2: Transcribe audio
            segments = transcribe_audio(audio_file_path, whisper_model)
            if segments is None:
                print(f"Skipping saving for {url} due to transcription failure.")
                continue

            # Step 3: Save transcription to JSON
            # Use the stem of the audio file name as the base for the JSON file name
            save_transcription_to_json(segments, transcription_dir, audio_file_path.name)

        except Exception as e:
            print(f"An error occurred during processing of {url}: {e}")
        finally:
            # Step 4: Clean up temporary audio file
            if audio_file_path and audio_file_path.exists():
                try:
                    os.remove(audio_file_path)
                    print(f"Cleaned up temporary audio file: {audio_file_path}")
                except Exception as e:
                    print(f"Error cleaning up {audio_file_path}: {e}")

    print("\n--- All videos processed ---")

# --- Example Usage ---
if __name__ == "__main__":
    # IMPORTANT: Replace with your actual Google Sheet key
    # This key is from the original notebook: '1-gartYfd6OXdtw64O6ZxbVuxznH-D5vSydyf7_k7j_M'
    # Ensure your Google Sheet is publicly accessible or you have given permissions to the service account.
    # Also, ensure the first column is named "youtube Url"
    YOUR_GOOGLE_SHEET_KEY = '1-gartYfd6OXdtw64O6ZxbVuxznH-D5vSydyf7_k7j_M'

    main_workflow(YOUR_GOOGLE_SHEET_KEY)

    # You can now proceed to load the generated JSON files from /content/subtitles
    # and use them with your Gemini API call as discussed previously.

Loading Whisper model (turbo)... This may take a moment.
Whisper model loaded.
Authenticating with Google Sheets...
Found 2 YouTube URLs in the spreadsheet.

--- Processing video 1/2: https://www.youtube.com/watch?v=FFH3uQDk2yU ---
Successfully downloaded audio to: /content/audioFiles/Build_Anything_With_Grok_4_and_n8n_AI_Agents.wav
Transcribing audio: /content/audioFiles/Build_Anything_With_Grok_4_and_n8n_AI_Agents.wav
Detected language: English


100%|██████████| 63280/63280 [00:38<00:00, 1647.52frames/s]


Transcription complete for: /content/audioFiles/Build_Anything_With_Grok_4_and_n8n_AI_Agents.wav
Saving segments to: /content/subtitles/Build_Anything_With_Grok_4_and_n8n_AI_Agents_segments.json
Successfully saved segments to /content/subtitles/Build_Anything_With_Grok_4_and_n8n_AI_Agents_segments.json
Cleaned up temporary audio file: /content/audioFiles/Build_Anything_With_Grok_4_and_n8n_AI_Agents.wav

--- Processing video 2/2: https://www.youtube.com/watch?v=foEW387Y4rU ---
Successfully downloaded audio to: /content/audioFiles/Someone_Will_Get_Really_Rich_Doing_This.wav
Transcribing audio: /content/audioFiles/Someone_Will_Get_Really_Rich_Doing_This.wav
Detected language: English


100%|██████████| 40320/40320 [00:15<00:00, 2570.00frames/s]

Transcription complete for: /content/audioFiles/Someone_Will_Get_Really_Rich_Doing_This.wav
Saving segments to: /content/subtitles/Someone_Will_Get_Really_Rich_Doing_This_segments.json
Successfully saved segments to /content/subtitles/Someone_Will_Get_Really_Rich_Doing_This_segments.json
Cleaned up temporary audio file: /content/audioFiles/Someone_Will_Get_Really_Rich_Doing_This.wav

--- All videos processed ---
